<a href="https://colab.research.google.com/github/gaaithri/aiml/blob/main/skillbuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

saishri_social_bk_path = kagglehub.dataset_download('saishri/social-bk')
saishri_eco_q1_path = kagglehub.dataset_download('saishri/eco-q1')
saishri_audio_ws_path = kagglehub.dataset_download('saishri/audio-ws')
saishri_testdo_path = kagglehub.dataset_download('saishri/testdo')
saishri_moneyfile_path = kagglehub.dataset_download('saishri/moneyfile')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install transformers

In [ ]:
!pip install sentence-transformers pyttsx3 git+https://github.com/openai/whisper.git


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-k32dj60_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-k32dj60_
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803670 sha256=b9cdf0956c8e130ed181e38410b8cb3de31aa8c288134ca7612826ef9b5efdc1
  Stored in directory: /tmp/pip-ephem-wheel-cache-ehgl6wtg/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.2 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pyttsx3
import whisper


<!-- -->
### Read the text input

In [ ]:
import PyPDF2
import os
pdf_directory = "/kaggle/input/social-bk/"
def read_pdfs(file_path):
    """
    Reads a PDF file and extracts text from it.
    """
    all_text = {}
    for file_name in os.listdir(pdf_directory):
        if file_name.endswith('.pdf'):  # Ensure only PDFs are processed
            file_path = os.path.join(pdf_directory, file_name)
            pdf_reader = PyPDF2.PdfReader(file_path)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            all_text[file_name] = text
    return all_text

# Extract text from all PDFs
pdf_texts = read_pdfs(pdf_directory)

#### Chunk inout

In [ ]:
def chunk_texts(pdf_texts, chunk_size=300):
    """
    Splits text from multiple PDFs into smaller chunks.
    """
    all_chunks = []
    chunk_mapping = {}  # To map chunks back to their source files
    for file_name, text in pdf_texts.items():
        words = text.split()
        chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
        all_chunks.extend(chunks)
        chunk_mapping.update({chunk: file_name for chunk in chunks})
    return all_chunks, chunk_mapping

chunks, chunk_mapping = chunk_texts(pdf_texts)


### Embedding Model  = recognising vocabulary

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_chunk_embeddings(chunks):
    """
    Generates embeddings for each chunk using SentenceTransformer.
    """
    return embedding_model.encode(chunks)

chunk_embeddings = generate_chunk_embeddings(chunks)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

#### Building hugging face pipline

In [ ]:
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


#### Finidng Answerable Chunk

In [ ]:
def find_relevant_chunk(question, chunks, chunk_embeddings):
    """
    Finds the most relevant chunk for a given question.
    """
    question_embedding = embedding_model.encode([question])[0]
    similarities = cosine_similarity([question_embedding], chunk_embeddings)
    best_chunk_index = np.argmax(similarities)
    return chunks[best_chunk_index]

def answer_question(question, chunks, chunk_embeddings):
    """
    Answers a question using the Q&A pipeline and the most relevant chunk.
    """
    relevant_chunk = find_relevant_chunk(question, chunks, chunk_embeddings)
    result = qa_pipeline(question=question, context=relevant_chunk)
    # Get the file name associated with the relevant chunk
    source_file = chunk_mapping[relevant_chunk]
    return result['answer'], source_file





In [ ]:
# Step 5: Test Q&A for Multiple PDFs
question = "Types of Money?"
answer, source_file = answer_question(question, chunks, chunk_embeddings)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(answer, source_file)

demand deposits jess203.pdf


In [ ]:



# Step 5: Voice Interaction (Optional)
def voice_interactive_bot(audio_file_path, chunks, chunk_embeddings):
    """
    Voice-interactive bot that takes audio input, answers the question, and speaks the answer.
    """
    # Load Whisper model for speech-to-text
    stt_model = whisper.load_model("base")

    # Convert speech to text
    result = stt_model.transcribe(audio_file_path)
    question = result['text']

    print(f"User Question: {question}")

    # Answer the question
    answer = answer_question(question, chunks, chunk_embeddings)
    print(f"Bot Answer: {answer}")

    # Convert text to speech
    tts_engine = pyttsx3.init()
    tts_engine.say(answer)
    tts_engine.runAndWait()

# Example Usage
# Uncomment and replace 'path_to_audio.wav' with your audio file path
# voice_interactive_bot('path_to_audio.wav', chunks, chunk_embeddings)

# Step 6: Manual Q&A Testing
# Example question
# question = "What is Artificial Intelligence?"
# answer = answer_question(question, chunks, chunk_embeddings)

# print(f"Question: {question}")
# print(f"Answer: {answer}")

In [ ]:
!pip install pydub
!apt-get install ffmpeg -y


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 132 not upgraded.


In [ ]:
!apt-get update -y
!apt-get install espeak -y

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]                
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease                                              
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]                 
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,631 kB]
Get:11 h

In [ ]:
!espeak "Hello, this is a test of eSpeak!"


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No

In [ ]:
!pip install gTTS

In [ ]:
from gtts import gTTS
import IPython.display as ipd

# Text to synthesize
text = "Hello, this is a test of Google Text-to-Speech!"

# Generate speech
tts = gTTS(text)
tts.save("output.mp3")

# Play audio in Kaggle
ipd.Audio("output.mp3")

In [ ]:
!pip install SpeechRecognition


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 56.1 MB/s eta 0:00:00:00:0100:01


In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2doa5h87
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2doa5h87
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
import whisper

# Load the Whisper model
model = whisper.load_model("base")  # Use "tiny", "base", "small", "medium", or "large" models

# Transcribe the audio file
def transcribe_audio(audio_file):
    """
    Transcribes audio to text using Whisper.
    """
    result = model.transcribe(audio_file)
    text = result["text"]
    print(f"Transcription: {text}")
    return text

# Example usage
audio_file = "/kaggle/input/testdo/MoneyQ1.wav"  # Ensure the file is in WAV format
transcription = transcribe_audio(audio_file)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 184MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription:  What is money and what are different types of money?


In [ ]:
!pip install gTTS


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
 from gtts import gTTS
import IPython.display as ipd

def generate_response(user_question):
    """
    Dummy response generator (replace with your custom logic).
    """
    return f"I heard: '{user_question}'. This is my response!"

def text_to_speech(text, output_file="response.mp3"):
    """
    Converts text to speech using gTTS and plays the audio.
    """
    tts = gTTS(text)
    tts.save(output_file)
    return ipd.Audio(output_file)

# Workflow: Transcribe -> Generate Response -> TTS
user_question = transcribe_audio(audio_file)
bot_response = generate_response(user_question)
audio_output = text_to_speech(bot_response)
ipd.display(audio_output)


Transcription:  What is money and what are different types of money?


In [ ]:
voice_interactive_bot('/kaggle/input/testdo/MoneyQ1.wav', chunks, chunk_embeddings)


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


User Question:  What is money and what are different types of money?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Bot Answer: ('demand deposits', 'jess203.pdf')


In [ ]:


!pip install pyttsx3


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
import os

source_file = "/kaggle/input/audio-ws/Final Agenda - BV convocation 2025.wav"
destination_file = "/kaggle/input/audio-ws/audio.wav"

try:
    os.rename(source_file, destination_file)
    print(f"File renamed from {source_file} to {destination_file}")
except FileNotFoundError:
    print(f"Error: Source file {source_file} not found.")
except FileExistsError:
    print(f"Error: Destination file {destination_file} already exists.")
except OSError as e:
    print(f"An OS error occurred: {e}")

An OS error occurred: [Errno 30] Read-only file system: '/kaggle/input/audio-ws/Final Agenda - BV convocation 2025.wav' -> '/kaggle/input/audio-ws/audio.wav'
